## Importing required libraries

In [ ]:
import os
from tqdm import tqdm
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from tensorflow.keras.utils import load_img
from tensorflow.keras.layers import Dense, Flatten,Dropout,ReLU,BatchNormalization,Softmax
from tensorflow.keras.models import Model # Create a output sequence
from tensorflow.keras.preprocessing.image import ImageDataGenerator # Augmantation
from tensorflow.keras.applications.efficientnet import EfficientNetB7 #Make infrance from the model
from tensorflow.keras.preprocessing import image
# from keras.utils import plot_model  #Visualize Model
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from keras.optimizers import RMSprop
from tensorflow.keras.utils import image_dataset_from_directory

In [ ]:
augmenter = ImageDataGenerator(rotation_range=30, horizontal_flip=True,width_shift_range=0.1,
                               height_shift_range=0.1,shear_range=0.2,zoom_range=0.2,fill_mode='nearest')

In [ ]:
gen = augmenter.flow_from_directory('/content/drive/MyDrive/project deep 1/data',(255,255),"rgb")

Found 3960 images belonging to 6 classes.


In [ ]:
def build_network(base_model, classes):
    """
    build a new network from a pre-trained model

    parameters:
        -base_model: pretraind model
        -classes: number of classes to classify

    """
    x = Flatten()(base_model.output)
    x = Dense(units=256)(x)
    x = ReLU()(x)
    x = BatchNormalization()(x)
    x = Dropout(rate=0.5)(x)
    x = Dense(units=classes)(x)
    output = Softmax()(x)
    return output


def create_model(CLASSES,input_shape = (500,500,3)):
  """
  Load the EfficientNetB7 models without the classifier part of the model.
  Freeze all layers, and adding output layers and train it.

  -------
  param:
    input_shape: image shape, It should have exactly 3 inputs channels.
    num_classes: number of classes to classify
  """

  # download the model and it's weights
  base_model = EfficientNetB7(weights='imagenet',include_top=False,input_shape=input_shape)
  # freeze layers
  for layer in base_model.layers:
    layer.trainable = False

  model = build_network(base_model, CLASSES)
  model = Model(base_model.input, model)

  return model

model = create_model(6,(255,255,3))

258076736/258076736 [==============================] - 2s 0us/step


In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=["accuracy"])

In [ ]:
model.fit_generator(gen,3960 // 64,20)

Epoch 1/20
61/61 [==============================] - 1658s 26s/step - loss: 1.3547 - accuracy: 0.5886
Epoch 2/20
61/61 [==============================] - 1542s 25s/step - loss: 0.8575 - accuracy: 0.7018
Epoch 3/20
61/61 [==============================] - 1528s 25s/step - loss: 0.7330 - accuracy: 0.7521
Epoch 4/20
61/61 [==============================] - 1529s 25s/step - loss: 0.6888 - accuracy: 0.7505
Epoch 5/20
61/61 [==============================] - 1536s 25s/step - loss: 0.6337 - accuracy: 0.7751
Epoch 6/20
61/61 [==============================] - 1573s 26s/step - loss: 0.5754 - accuracy: 0.7925
Epoch 7/20
61/61 [==============================] - 1543s 25s/step - loss: 0.5320 - accuracy: 0.8212
Epoch 8/20
61/61 [==============================] - 1511s 25s/step - loss: 0.4816 - accuracy: 0.8344
Epoch 9/20
61/61 [==============================] - 1551s 25s/step - loss: 0.4653 - accuracy: 0.8325
Epoch 10/20
61/61 [==============================] - 1515s 25s/step - loss: 0.4515 - accura

In [ ]:
model.save('/content/drive/MyDrive/project deep 1/model #1.h5')